In [ ]:
from skbio import TreeNode
from qiime2 import Artifact
import pandas as pd
from qiime2.plugins import empress
from qiime2 import Visualization
import numpy as np
from scipy import stats

# Import data

In [ ]:
# covid feature table
covid_table = pd.read_csv("fig2a/input/COVID19-v2/COVID_prromenade.csv", index_col="Feature")
covid_table[covid_table == 1] = 0

# feature metadata
f_change = pd.read_csv("fig2a/input/COVID_featureChanges.csv", index_col="feature id", delimiter='\t')
f_text = pd.read_csv("fig2a/input/feature-metadata.tsv", index_col="feature-id", delimiter="\t")

# Calculate ks p-values for healthy control

In [ ]:
# calculate covid_hc_sig (covid vs healthy control ks p_value) for each feature
covid_hc_pval = {}
columns = covid_table.T.columns.to_list()
for column in columns:
    covid = [] # stores feature observations for covid samples
    hc = [] # stores feature observations for healthy control samples
    
    # iterate over each feature
    for key, val in covid_table.T[column].to_dict().items():
        # add feature observations to arrays 
        if "COVID" in key:
            covid.append(val)
        elif "HC" in key:
            hc.append(val)
    # get p-value from ks test
    covid_hc_pval[column] = stats.ks_2samp(np.array(covid), np.array(hc))[1]

# Calculate ks p-values for pneumonia patients

In [ ]:
# calculate covid_hc_sig (covid vs pneumonia patients ks p_value) for each feature
covid_pn_pval = {}
columns = covid_table.T.columns.to_list()
for column in columns:
    covid = [] # stores feature observations for covid samples
    pn = [] # stores feature observations for pneumonia patient samples
    
    # iterate over each feature
    for key, val in covid_table.T[column].to_dict().items():
        # add feature observations to arrays 
        if "COVID" in key:
            covid.append(val)
        elif "CAP" in key:
            pn.append(val)
    # get p-value from ks test
    covid_pn_pval[column] = stats.ks_2samp(np.array(covid), np.array(pn))[1]

# Find significant features

In [ ]:
# marks a feature as significant if its ks p-value is < 0.05
def assign_sig(val):
    if val < 0.05:
        return True
    return False
covid_hc_pval_rank = {key:assign_sig(val) for key, val in covid_hc_pval.items()}
covid_pn_pval_rank = {key:assign_sig(val) for key, val in covid_pn_pval.items()}

# Create tree

In [ ]:
# read node list file
node_list = open("fig2a/input/nodes_prromenade.txt").read().replace("\t", "").split('\n')
node_list.pop() # remove the EOF entry
c_to_p = node_list
c_to_p = [child_parent.split("|") for child_parent in c_to_p]
c_to_p = [(child_parent[0], child_parent[1]) for child_parent in c_to_p] # create (child, parenent)

# rename nodes to use EC_code
id_to_EC = open("fig2a/input/kegg_bactLTU_virusLTU.taxid_code_name.txt").read().split("\n")
id_to_EC = id_to_EC[1:]
id_to_EC.pop()
id_to_EC = [i.split("\t")[:2] for i in id_to_EC] # keep the first two columns
id_to_EC = {i[0]: i[1] for i in id_to_EC}

# create tree
tree = {id_to_EC["1"]: TreeNode(id_to_EC["1"], length=1)} # root node
for child, parent in c_to_p:
    child = id_to_EC[child]
    parent = id_to_EC[parent]
    if child not in tree:
        tree[child] = TreeNode(child, length=1, parent=tree[parent]) # link child to parent
    tree[parent].children.append(tree[child]) # link parent to child
    
# get root of tree
tree = tree[id_to_EC["1"]]

# get list of tips
tips = []
for node in tree.postorder():
    if node.is_tip():
        tips.append(node.name)

# Find which features are more/less significantly abundant in COVID-19

In [ ]:
# grab features in preorder in preorder
names = [node.name for node in tree.preorder(include_self=True)]
f_text = f_text.loc[names]

features = id_to_EC.values()
taxes = f_text.loc[features, "text_description"].to_list()
covid_hc_mean_map = f_change["COVID_minus_HC"].to_dict()
covid_pn_mean_map = f_change["COVID_minus_CAP"].to_dict()
for node, tax in zip(features, taxes):
    # get node
    t_node = tree.find(node)
    
    # assign taxonomy
    t_node.tax = tax
    
    # assign p-value (hc)
    if node in covid_hc_pval:
        t_node.covid_hc_pval = covid_hc_pval[node]
    else:
        t_node.covid_hc_pval = 1
        
    # assign p-value (pn)
    if node in covid_pn_pval:
        t_node.covid_pn_pval = covid_pn_pval[node]
    else:
        t_node.covid_pn_pval = 1
    
    # assign abundance value (hc)
    if node in covid_hc_pval:
        if covid_hc_pval_rank[node]: # feature is significant
            if covid_hc_mean_map[node] > 0: # more abundant in COVID-19
                t_node.covid_hc_is_sig = 1
            else: # less abundant in COVID-19
                t_node.covid_hc_is_sig = -1
        else:
            t_node.covid_hc_is_sig = 0 #no significant difference
    else: # no significant difference
        t_node.covid_hc_is_sig = 0
    
    # assign abundance value (pn)
    if node in covid_pn_pval:
        if covid_pn_pval_rank[node]: # feature is significant
            if covid_pn_mean_map[node] > 0: # more abundant in COVID-19
                t_node.covid_pn_is_sig = 1
            else: # less abundant in COVID-19
                t_node.covid_pn_is_sig = -1
        else:
            t_node.covid_pn_is_sig = 0 # no significant difference
    else: # no significant difference
        t_node.covid_pn_is_sig = 0
        
# project taxonomy up tree
for node in tree.preorder(include_self=True):
    level = len(node.ancestors())
    setattr(node, "level_" + str(level), node.tax)
    for i in range(1, level):
        tax = getattr(node.parent, "level_" + str(i))
        setattr(node, "level_" + str(i), tax)

# add taxonomy to metadata
for i in range(1, 5):
    f_text["level_" + str(i)] = [getattr(node, "level_" + str(i)) if hasattr(node, "level_" + str(i)) \
                                    else "" for node in tree.preorder(include_self=True)]

# Create feature metadata file

In [ ]:
columns = f_text.index.to_list()
f_text["p_value"] = [tree.find(name).covid_hc_pval for name in columns]
f_text["covid_hc_sig"] = [tree.find(name).covid_hc_is_sig for name in columns]
f_text["covid_pn_sig"] = [tree.find(name).covid_pn_is_sig for name in columns]
f_meta = f_change.merge(f_text, left_index=True, right_index=True,how="right")
f_meta["custom_level"] = f_meta["level_1"].copy()
f_meta.loc[f_meta["custom_level"] == "Lyases", "custom_level"] = \
        f_meta.loc[f_meta["custom_level"] == "Lyases", "level_2"]
f_meta.to_csv("fig2a/output/feature-change-metadata.tsv", sep="\t")

# Create qiime artifacts

In [ ]:
f = covid_table.index.to_list()
f_k = [i for i in f if i in tips]
covid_table = covid_table.loc[f_k].T
Artifact.import_data("FeatureTable[Frequency]", covid_table).save("fig2a/output/covid-table.qza")
Artifact.import_data("Phylogeny[Rooted]", tree, view_type=TreeNode).save("fig2a/output/tree.qza")

# Create EMPress plot

In [ ]:
# create visualization
!qiime empress community-plot \
    --i-tree fig2a/output/tree.qza \
    --i-feature-table fig2a/output/covid-table.qza \
    --m-sample-metadata-file fig2a/input/ciaa203_suppl_supplementary_tables_s1-s6.csv \
    --m-feature-metadata-file fig2a/output/feature-change-metadata.tsv \
    --o-visualization fig2a/output/covid-plot-no-emperor.qzv --p-no-filter-missing-features

# View EMPress plot

In [ ]:
# view tree visualization
Visualization.load("fig2a/output/covid-plot-no-emperor.qzv")